In [ ]:
#import scriptFile as sF, shutil,  visualize as funcVis, importlib as impL, os, projRelatedHelperFuncs as prHF
import torch
import numpy as np
print(torch.__version__)
from collections import Counter
import pandas as pd 
import os
import importlib as impL

In [ ]:
import sys
sys.path.append("/home/doga/GithUBuntU/keyhandshapediscovery")
#sys.path.extend(['/home/doga/GithUBuntU/keyhandshapediscovery'])
import helperFuncs as funcH
import dataLoaderFuncs as funcD
import projRelatedHelperFuncs as prHF #prepare_data_4
import dataLoaderFuncs as dlF #HandCraftedDataset
import modelFuncs as moF #MLP
import scriptFile as sf
impL.reload(funcH)
impL.reload(funcD)
impL.reload(prHF)
impL.reload(dlF)
impL.reload(moF)
impL.reload(sf)
funcH.setPandasDisplayOpts()

In [ ]:
nos = 11
labelNames = prHF.load_label_names(nos)
results_dir = funcH.getVariableByComputerName('results_dir')  # '/media/dg/SSD_Data/DataPath/bdResults'
baseLineResultFolder = os.path.join(results_dir, 'baseResults')  # '/media/dg/SSD_Data/DataPath/bdResults/baseResults'
baseResFiles = funcH.getFileList(baseLineResultFolder, startString="", endString="_"+str(nos)+"_KMeans_256.npz", sortList=False)
for f in baseResFiles:
    labels_true, labels_pred = prHF.loadBaseResult(f)
    labels_true, labels_pred, _ = funcH.getNonZeroLabels(labels_true, labels_pred)
    _confMat_preds, kluster2Classes, kr_pdf = funcH.countPredictionsForConfusionMat(labels_true, labels_pred, labelNames=labelNames)
    sampleCount = np.sum(np.sum(_confMat_preds))
    acc = 100 * np.sum(np.diag(_confMat_preds)) / sampleCount
    print(f, labels_true.shape, labels_pred.shape, acc)
    print(kr_pdf)
    print("****")

In [ ]:
dataIdent="hgsk"
nos=11
pca_dim=128
verbose=0
ft, lb, lb_sui, lb_map = prHF.combine_pca_hospisign_data(dataIdent=dataIdent, nos=nos, pca_dim=pca_dim, verbose=verbose)
class_names = np.asarray(lb_map["khsName"])

In [ ]:
impL.reload(funcH)
impL.reload(dlF)
impL.reload(moF)
impL.reload(sf)
impL.reload(prHF)
resultDict = prHF.runClusteringOnFeatSet_Aug2020(ft, lb, lb_map, dataToUse=dataIdent, numOfSigns=nos, pcaCount=pca_dim, clustCntVec=[256])

In [ ]:
impL.reload(prHF)
prHF.traverseBaseResultsFolder_Aug2020()

In [ ]:
impL.reload(prHF)
prHF.getBaseResults_Aug2020(dataToUse="hog", pcaCount=256, numOfSigns=11, displayResults=True, baseResultFileName='')

In [ ]:
print(class_names)

In [ ]:
impL.reload(prHF)
labelNames, labels, predictionsDict, cluster_runs, N = prHF.load_labels_pred_for_ensemble_Aug2020(class_names)
prHF.ensemble_cluster_analysis(cluster_runs, predictionsDict, labels,
                     consensus_clustering_max_k=256, useNZ=False, nos=11,
                     resultsToCombineDescriptorStr="hogsnsk",
                     labelNames = labelNames, verbose=True)

In [ ]:
x = np.zeros((5, 5), dtype=np.float32)

In [ ]:
data_ident = "sk"
ft, lb, lb_sui, lb_map = prHF.combine_pca_hospisign_data(data_ident, pca_dim=96, nos=41, verbose=0)
uniqLabs = np.unique(lb)
classCount = len(uniqLabs)

In [ ]:
print("uniqLabs=", uniqLabs, ", classCount_="+ data_ident, classCount)
class_names = np.asarray(lb_map["khsName"])
print(class_names)

In [ ]:
cntVec = []
for label in list(uniqLabs):
    inds = funcH.getInds(np.array(lb), label)
    cnt = len(inds)
    print(label, class_names[label], cnt)
    cntVec.append(cnt)
    #khs_cnt_df.append({'khsID':label,'khsCnt':cnt})

In [ ]:
khs_cnt_df = pd.DataFrame({'khsName':class_names, 'khsCnt':cntVec})
khs_cnt_df = khs_cnt_df.sort_values(by=['khsCnt'], ascending=False)
print(khs_cnt_df)

In [ ]:
khs_cnt_df.to_csv('/home/doga/Desktop/khsCnt_67Class.csv')

In [ ]:
def get_hospisign_labels(nos=11, sortBy=None, verbose=0):
    base_dir = funcH.getVariableByComputerName('base_dir')
    baseFold = os.path.join(base_dir, "neuralNetHandImages_nos" + str(nos) + "_rs224", "imgs")
    list_dict_file = os.path.join(baseFold, "list_dict.txt")
    a = pd.read_csv(list_dict_file, delimiter="*", header=None,
                    names=["sign", "user", "rep", "frameID", "khsID", "khsName", "hand"])
    b, uniqKHSinds = np.unique(np.asarray(a["khsID"]), return_index=True)
    labelsAll = np.asarray(a["khsID"], dtype=int)
    namesAll = np.asarray(a["khsName"])

    labels_sui = np.squeeze(np.asarray(a[["sign", "user", "khsID"]]))
    # get sort index first
    assignedKHSinds = labelsAll[uniqKHSinds]
    selectedKHSnames = np.array([str(np.char.strip(n)) for n in namesAll[uniqKHSinds]])

    if verbose > 1:
        print(labelsAll.shape, labelsAll.dtype)
    sortedLabelsAll, sortedLabelsMap = funcH.reset_labels(labelsAll, assignedKHSinds, selectedKHSnames, sortBy=sortBy,
                                                          verbose=verbose)
    if verbose > 1:
        print("sortedLabelsAll:\n", sortedLabelsAll.head())
        print("sortedLabelsMap:\n", sortedLabelsMap)
        print(labels_sui.shape, labels_sui.dtype)
    labels_sui[:, 2] = np.squeeze(np.array(sortedLabelsAll))

    lb_map = np.vstack((sortedLabelsMap["labelIDs"], sortedLabelsMap["labelStrings"])).T

    x = Counter(np.squeeze(labelsAll).astype(int))

    khsCntVec = [v for k, v in x.most_common()]
    khsIndex = [k for k, v in x.most_common()]
    if verbose > 2:
        print("x:\n", x)
        khsNameCol = [str(np.squeeze(lb_map[np.where(lb_map[:, 0] == k), 1])) for k, v in x.most_common()]
        print("khsNameCol:\n", khsNameCol)
        print("khsCntVec:\n", khsCntVec)
        print("khsIndex:\n", khsIndex)
    khsCntCol = np.asarray(khsCntVec)[np.argsort(khsIndex)]
    if verbose > 2:
        print("khsCntVec(sorted accordingly):\n", khsCntCol)

    lb_map_new = pd.DataFrame({"khsID": lb_map[:, 0], "khsName": lb_map[:, 1], "khsCnt": khsCntCol})
    lb_map_cnt = lb_map_new.sort_values(by='khsCnt', ascending=False)
    lb_map_id = lb_map_new.sort_values(by='khsID', ascending=True)
    lb_map_name = lb_map_new.sort_values(by='khsName', ascending=True)
    if verbose > 1:
        print("lb_map_cnt=\n", lb_map_cnt)
        print("lb_map_id=\n", lb_map_id)
        print("lb_map_name=\n", lb_map_name)

    hospisign_labels = {
        "labels": sortedLabelsAll,
        "labels_sui": labels_sui,
        "khsInds": sortedLabelsMap["labelIDs"],
        "khsNames": sortedLabelsMap["labelStrings"],
        "label_map": lb_map_id,
        "label_map_cnt": lb_map_cnt,
        "label_map_name": lb_map_name,
    }

    return hospisign_labels

In [ ]:
hospisign_labels = get_hospisign_labels(nos=41, sortBy=None, verbose=0)